# **INSURANCE ANALITYCS: TRAVEL INSURANCE CLAIM**

**CREATED BY: PRIMA ADE SUKRONO**

## **BUSINESS PROBLEM UNDERSTANDING**

<u>**Context**</u>

Asuransi perjalanan adalah jenis asuransi yang memberikan perlindungan selama kita bepergian, baik di dalam negeri maupun luar negeri. Beberapa negara bahkan mewajibkan wisatawan untuk memiliki asuransi perjalanan, misalnya negara-negara di Eropa dan Amerika. Besarnya premi bergantung pada cakupan yang diinginkan, lama perjalanan, dan tujuan perjalanan. Perusahaan asuransi perjalanan ingin mengetahui pemegang polis yang akan mengajukan klaim asuransi. Data pemegang polis di perusahaan asuransi merupakan data historis yang terdiri dari destinasi, produk asuransi, dan sebagainya.

Target:
- 0: Tidak mengajukan claim (No)
- 1: Mengajukan claim (Yes)

---

<u>**Problem Statement**</u>

Prediksi risiko secara manual masih dapat dilakukan oleh tim Underwriting, namun proses ini cenderung memakan waktu lebih lama dan memiliki potensi kesalahan yang lebih tinggi. Keterbatasan dalam memproses kombinasi fitur secara kompleks secara manual dapat menyebabkan kesalahan prediksi—khususnya `False Negative`, yaitu calon nasabah dengan risiko tinggi klaim yang tidak terdeteksi.

Jika pendekatan konservatif digunakan, di mana semua calon nasabah diasumsikan akan mengajukan klaim, maka akan terjadi ketidakadilan dalam penetapan premi. Nasabah berisiko rendah diperlakukan sama dengan yang berisiko tinggi, yang dapat menurunkan minat pembelian polis baru dan mendorong nasabah loyal untuk berpindah ke kompetitor yang menawarkan skema premi lebih adil.

Untuk meningkatkan efisiensi operasional dan profitabilitas, tim Underwriting memerlukan sistem prediktif berbasis machine learning yang mampu mengidentifikasi calon nasabah dengan risiko klaim tinggi secara lebih akurat, serta memberikan wawasan terhadap faktor-faktor utama yang memengaruhi risiko tersebut. Sistem ini diharapkan mendukung implementasi strategi premi berbasis resiko (risk-based pricing), sehingga penetapan premi dapat lebih tepat sasaran, mengurangi kerugian finansial akibat klaim tak terduga, dan memperkuat daya saing perusahaan di pasar asuransi.

---

<u>**Tujuan Analisis**</u>

1. Memprediksi risiko klaim setiap calon pemegang polis berdasarkan data historis seperti durasi perjalanan, produk asuransi, tujuan perjalanan, usia, dan lainnya.
2. Mengidentifikasi faktor-faktor utama (risk drivers) yang menyebabkan nasabah cenderung mengajukan klaim, sebagai dasar untuk:
   - Penyesuaian premi individu (risk-based pricing)
   - Desain ulang produk dan strategi distribusi
   - Segmentasi pelanggan
3. Mendukung strategi penetapan harga yang adaptif dengan mempertimbangkan tingkat risiko yang diprediksi oleh model.
4. Membantu tim underwriting dan tim terkait membuat keputusan yang lebih akurat terhadap nasabah berisiko tinggi melalui rekomendasi berbasis data.

--- 

<u>**Pendekatan Analitik**</u>

Kita akan melakukan analisa dari data historis nasabah untuk mengidentifikasi pola yang membedakan antara nasabah yang mengajukan klaim dan yang tidak, lalu membangun model machine learning untuk memprediksi risiko klaim secara otomatis.

<u>**Skenario Awal**</u>

Berdasarkan data awal, jika diasumsikan prediksi claim hanya berdasarkan durasi perjalanan lebih dari 30 hari:

|Actual\\\Prediksi  |Prediksi Negative          |Prediksi Positif              | 
|:-----------------:|:-------------------------:|:----------------------------:|
|Actual Negative    |TN (26,067)                |FP (17,584) * Cost (100 USD)  |
|Actual Positive    |FN (312) * Cost (305 USD)  |TP (365)                      |

Maka kerugian yang dapat ditimbulkan adalah:
- FN = 312 * 305    =   95,160 USD
- FP = 17,584 * 100 =   1,758,400 USD
- **Total**         =   1,853,560 USD

<u>**Metric Evaluasi**</u>

Jika dilihat dari sisi biaya yang ditimbulkan oleh kesalahan prediksi (*bukan total biaya keseluruhan*), maka kesalahan dalam **tidak mendeteksi nasabah yang sebenarnya mengajukan klaim (FN) memiliki dampak biaya yang jauh lebih besar** dibandingkan dengan salah memprediksi nasabah yang sebenarnya tidak mengajukan klaim (FP).

Berdasarkan pertimbangan tersebut, model yang dibangun akan difokuskan untuk semaksimal mungkin mendeteksi nasabah yang benar-benar akan mengajukan klaim (TP), serta meminimalkan kesalahan prediksi pada nasabah yang berisiko tinggi untuk claim (FN), tanpa sepenuhnya mengabaikan tingkat kesalahan dalam memprediksi nasabah yang tidak mengajukan claim (FP).

Oleh karena itu, metrik evaluasi utama yang akan digunakan adalah **Recall**, karena recall mencerminkan kemampuan model dalam menangkap sebanyak mungkin kasus klaim yang benar.

## **DATA UNDERSTANDING & CLEANING**

In [1]:
# Import libraries

# General libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# data splitting
from sklearn.model_selection import train_test_split

# impute missing values
from sklearn.impute import SimpleImputer

# preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.preprocessing import OneHotEncoder
from category_encoders import OrdinalEncoder, BinaryEncoder
from feature_engine.discretisation import ArbitraryDiscretiser

# column transformer & pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline

# cross validation
from sklearn.model_selection import cross_val_score

# algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

# evaluation metric
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Gridsearch
from sklearn.model_selection import GridSearchCV

### 1. Load Dataset

In [2]:
# Import dataset
df_travel = pd.read_csv('data_travel_insurance.csv')

### 2. Data Understanding

In [3]:
df_travel.head()

,Agency,Agency Type,Distribution Channel,Product Name,Gender,Duration,Destination,Net Sales,Commision (in value),Age,Claim
0,C2B,Airlines,Online,Annual Silver Plan,F,365,SINGAPORE,216.0,54.0,57,No
1,EPX,Travel Agency,Online,Cancellation Plan,NaN,4,MALAYSIA,10.0,0.0,33,No
2,JZI,Airlines,Online,Basic Plan,M,19,INDIA,22.0,7.7,26,No
3,EPX,Travel Agency,Online,2 way Comprehensive Plan,NaN,20,UNITED STATES,112.0,0.0,59,No
4,C2B,Airlines,Online,Bronze Plan,M,8,SINGAPORE,16.0,4.0,28,No


Setiap baris dalam dataset merepresentasikan satu transaksi penjualan polis asuransi perjalanan, di mana produk asuransi yang sama dapat terjual berkali-kali kepada nasabah yang berbeda atau untuk perjalanan yang berbeda.


Penjelasan setiap kolom dalam dataset:

- Agency: Nama dari agency.
- Agency Type: Jenis dari agen asuransi perjalanan.
- Distribution Channel: Channel distribusi dari agen asuransi perjalanan.
- Product Name: Nama dari produk asuransi perjalanan.
- Gender: Jenis kelamin dari pemegang polis.
- Duration: Lama waktu perjalanan. (*karena tidak disebutkan sebelumnya, maka diasumsikan ini dalam bentuk hari*)
- Destination: Tujuan perjalanan.
- Net Sales: Jumlah penjualan bersih dari polis asuransi perjalanan.
- Commission (in value): Komisi yang diterima agen dari hasil penjualan.
- Age: Umur dari pemegang polis.
- Claim: Status Claim.

In [4]:
df_travel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44328 entries, 0 to 44327
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Agency                44328 non-null  object 
 1   Agency Type           44328 non-null  object 
 2   Distribution Channel  44328 non-null  object 
 3   Product Name          44328 non-null  object 
 4   Gender                12681 non-null  object 
 5   Duration              44328 non-null  int64  
 6   Destination           44328 non-null  object 
 7   Net Sales             44328 non-null  float64
 8   Commision (in value)  44328 non-null  float64
 9   Age                   44328 non-null  int64  
 10  Claim                 44328 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 3.7+ MB


Total data ada 44,328 baris dan 11 kolom, dimana terlihat ada 1 kolom terdapat missing value yaitu `Gender`, dan Type data terlihat sudah sesuai dengan isi kolom.

In [5]:
df_travel.describe(include='all')

,Agency,Agency Type,Distribution Channel,Product Name,Gender,Duration,Destination,Net Sales,Commision (in value),Age,Claim
count,44328,44328,44328,44328,12681,44328.000000,44328,44328.000000,44328.000000,44328.000000,44328
unique,16,2,2,26,2,NaN,138,NaN,NaN,NaN,2
top,EPX,Travel Agency,Online,Cancellation Plan,M,NaN,SINGAPORE,NaN,NaN,NaN,No
freq,24656,32113,43572,12979,6504,NaN,9267,NaN,NaN,NaN,43651
mean,NaN,NaN,NaN,NaN,NaN,49.424292,NaN,40.550948,9.707692,39.925600,NaN
std,NaN,NaN,NaN,NaN,NaN,109.153961,NaN,48.661970,19.625637,13.954926,NaN
min,NaN,NaN,NaN,NaN,NaN,-1.000000,NaN,-357.500000,0.000000,0.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,9.000000,NaN,18.000000,0.000000,35.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,22.000000,NaN,26.500000,0.000000,36.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,53.000000,NaN,48.000000,11.550000,43.000000,NaN


Dari hasil deskripsi data diatas, ditemukan bahwa terdapat anomali pada isi dari beberapa kolom berikut:
- Duration: Minimum hari adalah -1
- Net Sales: Minimum penjualannya -357.5
- Commision: Minimum dan median-nya 0
- Age: Minimumnya 0 dan Maximumnya 118

Maka perlu dilakukan pengecekan lebih lanjut pada kolom-kolom tersebut pada tahap pengecekan missing value.

In [6]:
df_travel['Claim'].value_counts(normalize=True)

Claim
No     0.984727
Yes    0.015273
Name: proportion, dtype: float64

Proporsi data terlihat imbalance, dimana 98% data tersebar pada nasabah yang tidak claim (No), sedangkan hanya 2% persen data pada nasabah yang claim (Yes). Perlu dilakukan percobaan balancing data pada tahap imbalance treatment.

Kesimpulan dari data understanding ini adalah:
1. Setiap baris dalam dataset merepresentasikan satu transaksi penjualan polis asuransi perjalanan.
2. Terdapat 1 Kolom yang beberapa nilainya hilang, yaitu kolom `Gender`
3. Terdapat 4 kolom yang memiliki anomali pada isinya yang perlu dilakukan pengecekan lebih lanjut, yaitu kolom `Duration`, `Net Sales`, `Commission`, dan `Age`.
4. Proporsi data terlihat imbalance, dimana 98% tersebar pada target 'No' dan hanya 2% tersebar pada target 'Yes'. Perlu dilakukan percobaan imbalance treatment untuk membandingkan performa sebelum dan setelah balancing.

### 3. Data Cleaning

#### A. Missing Value

In [7]:
list_item = []
num_col = df_travel.select_dtypes(include='number').columns

for col in df_travel.columns :
    list_item.append([col, df_travel[col].isna().sum(), 
                      round((df_travel[col].isna().sum()/len(df_travel[col])) * 100, 2)])
    if col in num_col:
        list_item.append(df_travel[col].min(numeric_only=True),
                      df_travel[col].median(numeric_only=True),
                      df_travel[col].max(numeric_only=True)])

df_info = pd.DataFrame(data=list_
                       item, columns=['features', 'null', 'null_perc'])
df_info

SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' on line 8 (782983592.py, line 10)

## **EXPLORATORY DATA ANALYSIS (EDA)**

## **DATA PREPARATION**

### 1. Define Feature (X) & Target (y)

### 2. Data Splitting

### 3. Preprocessing

## **MODELLING & EVALUATION**

### 1. Cross Validation

### 2. Hyperparameter Tuning

### 3. Prediction

### 4. Evaluation